In [ ]:
import sys
sys.path.append('/Users/user/StockAnalysisInPython/05_Stock_Price_API/')
from Investar import Analyzer
mk = Analyzer.MarketDB()
# error : OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")
# grant all privileges on *.* to root@localhost identified BY 'passwords' with grant option;
# ref : https://stackoverflow.com/questions/10181344/mysql-exceptions-operationalerror-1045-access-denied-for-user-rootlocalh

In [ ]:
# !pip install pandas_datareader
# !pip install yfinance

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from slacker import Slacker
import time
from pandas_datareader import data as pdr 
import yfinance as yf
yf.pdr_override()

In [ ]:
# mk.get_daily_price() 하는 것과 비슷하게 구성해야함
stocks = ["NVDA","SQ","AMZN","FNGU","O","QQQ","TQQQ"]
df = pd.DataFrame()
for s in stocks:
    df[s] = pdr.get_data_yahoo(tickers = s,
                          start = '2020-01-01',
                          end = '2020-11-03',
                         auto_adjust=False)['Close']

In [ ]:
df.head()

In [ ]:
# 하나의 종목만 보고 싶을 때
# aapl = pdr.get_data_yahoo(tickers="AAPL",
#                           start = '2020-09-01',
#                           end = '2020-10-31',
#                          auto_adjust=False)
# aapl.head()

In [ ]:
daily_ret = df.pct_change() 
annual_ret = daily_ret.mean() * 252
daily_cov = daily_ret.cov() 
annual_cov = daily_cov * 252

In [ ]:
annual_cov

In [ ]:
port_ret = [] 
port_risk = [] 
port_weights = [] 
sharpe_ratio = [] 

In [ ]:
for _ in range(10000): 
    weights = np.random.random(len(stocks)) 
    weights /= np.sum(weights) 

    returns = np.dot(weights, annual_ret) 
    risk = np.sqrt(np.dot(weights.T, np.dot(annual_cov, weights))) 

    port_ret.append(returns) 
    port_risk.append(risk) 
    port_weights.append(weights)
    sharpe_ratio.append(returns/risk)  # ①

In [ ]:
portfolio = {'Returns': port_ret, 'Risk': port_risk, 'Sharpe': sharpe_ratio}
for i, s in enumerate(stocks): 
    portfolio[s] = [weight[i] for weight in port_weights] 
df = pd.DataFrame(portfolio) 
df = df[['Returns', 'Risk', 'Sharpe'] + [s for s in stocks]]  # ② 

In [ ]:
max_sharpe = df.loc[df['Sharpe'] == df['Sharpe'].max()]  # ③
min_risk = df.loc[df['Risk'] == df['Risk'].min()]  # ④

In [ ]:
max_sharpe # 최대수익 내는 조합

In [ ]:
min_risk # 최소손실 내는 조합

In [ ]:
df.plot.scatter(x='Risk', y='Returns', c='Sharpe', cmap='viridis',
    edgecolors='k', figsize=(11,7), grid=True)  # ⑤
plt.scatter(x=max_sharpe['Risk'], y=max_sharpe['Returns'], c='r', 
    marker='*', s=300)  # ⑥
plt.scatter(x=min_risk['Risk'], y=min_risk['Returns'], c='r', 
    marker='X', s=200)  # ⑦
plt.title('Portfolio Optimization') 
plt.xlabel('Risk') 
plt.ylabel('Expected Returns') 
plt.show()

In [ ]:
slack = Slacker('Masking Token')
now = time.localtime()
slack.chat.post_message('#korean-auto-trader-bot',
                        "%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec) + '\n optimal Portfolio를 완료되었습니다!\n'
                        + "%04d/%02d/%02d" % (now.tm_year, now.tm_mon, now.tm_mday) + '일 데이터 기준 test 포트폴리오(금일 스터디 내용 중) \n'
                        + '최대수익 조합일 때 수익은 ' + str(max_sharpe['Returns'].values*100) + '% 입니다 \n'
                        + '이 때 변동성은 '+ str(max_sharpe['Risk'].values*100) + '% 입니다 \n'
                        + '최소손실 조합일 때 수익은 ' + str(min_risk['Returns'].values*100) + '% 입니다 \n'
                        + '이 때 변동성은 '+ str(min_risk['Risk'].values*100) + '% 입니다 \n'
                        ,as_user=True)